# 2D segmentation task with FLaVor inference service

This guide will walk you through tailoring the FLaVor inference service for 2D segmentation tasks using the model from [lungmask](https://github.com/JoHof/lungmask).

## Prerequisite

As for the working environment, please ensure you have the following dependencies installed:

```
python > 3.8
torch > 1.13
lungmask == 0.2.18
```

or simply run:

In [ ]:
!poetry install --with examples --extras infer 

## Implementation

### Setup imports

In [ ]:
import os
from typing import Any, Sequence, Tuple

import numpy as np
import SimpleITK as sitk
from lungmask import LMInferer

from flavor.serve.apps import InferAPP
from flavor.serve.inference import (
    BaseAiCOCOInferenceModel,
    BaseAiCOCOInputDataModel,
    BaseAiCOCOOutputDataModel,
)
from flavor.serve.models import AiImage, InferCategory
from flavor.serve.strategies import AiCOCOSegmentationOutputStrategy

### Setup inference model

In this section, we would create `SegmentationInferenceModel` inheriting from `BaseAiCOCOInferenceModel`. There are few abstract methods that we must override such as `define_inference_network`, `set_categories`, `set_regressions`, `data_reader` and `output_formatter`. As for `preprocess`, `inference` and `postprocess`, it is optional but here we override them since we are executing a 3D model.

Firstly, we need to implement submethods: `define_inference_network`, `set_categories` and `set_regressions`. These are defined in the `__init__()` constructor of the parent class `BaseAiCOCOInferenceModel`. `define_inference_network` defines your inference network and loads its pre-trained weight. `set_categories` and `set_regressions` define category and regression information. For example, a segmentation output would contain `c` channels. We need to show the exact meaning of each channel by specifying in `set_categories`. Refer to the following example for more detail.

Next, we implement other submethods that would be used in the `__call__` function of our inference model. See below workflow.

### `__call__` function workflow for the inference model
![__call__](images/call.png "inference workflow")

In [ ]:
class SegmentationInferenceModel(BaseAiCOCOInferenceModel):
    def __init__(self):
        self.formatter = AiCOCOSegmentationOutputStrategy()
        super().__init__()

    def define_inference_network(self):
        return LMInferer(modelname="LTRCLobes", fillmodel="R231")

    def set_categories(self):
        categories = [
            {"name": "Background", "display": False},
            {"name": "Left Upper Lobe", "display": True},
            {"name": "Left Lower Lobe", "display": True},
            {"name": "Right Upper Lobe", "display": True},
            {"name": "Right Middle Lobe", "display": True},
            {"name": "Right Lower Lobe", "display": True},
        ]
        return categories

    def set_regressions(self):
        return None

    def data_reader(self, files: Sequence[str], **kwargs) -> Tuple[np.ndarray, None, None]:
        dicom_reader = sitk.ImageFileReader()
        dicom_reader.SetFileName(files[0])
        dicom_reader.ReadImageInformation()
        dicom = sitk.GetArrayFromImage(dicom_reader.Execute()).squeeze()

        return dicom, None, None

    def preprocess(self, data: np.ndarray) -> np.ndarray:
        data = np.expand_dims(data, axis=0)
        return data

    def inference(self, x: np.ndarray) -> np.ndarray:
        return self.network.apply(x)

    def postprocess(self, out: Any, metadata: Any = None) -> Any:
        # (1, h, w) -> (c, h, w)
        out = [
            np.expand_dims((out == i).astype(np.uint8), axis=0)
            for i in range(6)  # or len(self.categories)
        ]
        out = np.concatenate(out, axis=0)
        return out

    def output_formatter(
        self,
        model_out: np.ndarray,
        images: Sequence[AiImage],
        categories: Sequence[InferCategory],
        **kwargs
    ) -> Any:

        output = self.formatter(model_out=model_out, images=images, categories=categories)
        return output

### Integration with InferAPP
We could integrate our defined inference model with FLaVor `InferAPP`, a FastAPI application. To initiate the application, users have to define `input_data_model` and `output_data_model` which are the standard input and output structure for the service. Then, provide `infer_function` as the main inference operation. After initiate the service, `/invocations` API end point would be available to process the inference request. We encourge users to implement a stand-alone python script based on this jupyter notebook tutorial.

#### (Optional) to initiate application in jupyter notebook, you have to run the following block.

```python
# This block is only for jupyter notebook. You don't need this in stand-alone script.
import nest_asyncio
nest_asyncio.apply()
```

#### Initiate the service

In [ ]:
app = InferAPP(
    infer_function=SegmentationInferenceModel(),
    input_data_model=BaseAiCOCOInputDataModel,
    output_data_model=BaseAiCOCOOutputDataModel,
)

In [ ]:
app.run(port=int(os.getenv("PORT", 9111)))

### Send request
We can send request to the running server by `send_request.py` which opens the input files and the coresponding JSON file and would be sent via formdata. We expect to have response in AiCOCO format.

```bash
# pwd: examples/inference
python send_request.py -f test_data/seg/0.dcm -d test_data/seg/input.json
```

## Setup Dockerfile
In order to interact with other services, we have to wrap the inference model into a docker container. Here's an example of the dockerfile.

```dockerfile
FROM nvidia/cuda:12.2.2-runtime-ubuntu20.04

RUN apt-get update \
    && DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
        python3.9 \
        python3-pip \
    && ln -sf /usr/bin/python3.9 /usr/bin/python
    && DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends wget\

RUN pip install torch==2.1.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121 --default-timeout=1000
RUN pip install https://github.com/ailabstw/FLaVor/archive/refs/heads/release/stable.zip -U && pip install "flavor[infer]"
RUN pip install monai==0.2.18

WORKDIR /app

COPY your_script.py  /app/

CMD ["python", "your_script.py"]

```